In [1]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cuda:0,floatX=float32"
#import theano
#print(theano.config.device, theano.config.floatX)

import keras
import PIL
import sys
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import csv
import tensorflow as tf
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
#from keras.applications import resnet
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15633937186140049250
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14395410420640952299
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 28437823488
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10516865573576874452
physical_device_desc: "device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:01:00.0, compute capability: 7.0"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7287621505262686273
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
K.set_learning_phase(1)

In [4]:
class_names = ['1', '2', '3', '4']
print(len(class_names))

4


In [5]:
img_width, img_height = 71, 71
nb_train_samples = 30451
nb_validation_samples = 7434
epochs = 12
batch_size = 8
n_classes = 4

In [6]:
import cv2
im = cv2.imread('home/z_andrei/zoohackathon/i_cropped_train/1/32886.jpg')
print(im)

None


In [7]:
train_data_dir = "/home/z_andrei/zoohackathon/i_cropped_train"    # дальше там папки, а не сами фотографии!!!!
validation_data_dir = "/home/z_andrei/zoohackathon/i_cropped_test"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')


Found 30451 images belonging to 4 classes.
Found 7434 images belonging to 4 classes.


In [9]:
def build_model_Xception():
  base_model = xception.Xception(include_top=True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None,
                                      classes = 4)
  
  for layer in base_model.layers:
    layer.trainable = True
      
  model = Model(inputs = base_model.input, outputs = base_model.output)
  return model

In [10]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse'])

In [12]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepath="weights-zoo-71-{epoch:02d}-{val_acc:.2f}.hdf5"
check = ModelCheckpoint(filepath, monitor = "val_acc", save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, check]

In [13]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size )

Epoch 1/12
3806/3806 [==============================] - 390s 102ms/step - loss: 0.4779 - acc: 0.8398 - mse: 0.0582 - val_loss: 0.2864 - val_acc: 0.8898 - val_mse: 0.0394
Epoch 2/12
3806/3806 [==============================] - 356s 94ms/step - loss: 0.3248 - acc: 0.8906 - mse: 0.0406 - val_loss: 0.7120 - val_acc: 0.9266 - val_mse: 0.0266
Epoch 3/12
3806/3806 [==============================] - 350s 92ms/step - loss: 0.2612 - acc: 0.9097 - mse: 0.0334 - val_loss: 0.0130 - val_acc: 0.9313 - val_mse: 0.0259
Epoch 4/12
3806/3806 [==============================] - 371s 98ms/step - loss: 0.2281 - acc: 0.9204 - mse: 0.0297 - val_loss: 0.0480 - val_acc: 0.9363 - val_mse: 0.0246
Epoch 5/12
3806/3806 [==============================] - 367s 96ms/step - loss: 0.1991 - acc: 0.9304 - mse: 0.0261 - val_loss: 0.0742 - val_acc: 0.9425 - val_mse: 0.0219
Epoch 6/12
3806/3806 [==============================] - 3202s 841ms/step - loss: 0.1835 - acc: 0.9359 - mse: 0.0241 - val_loss: 0.0529 - val_acc: 0.9444 -

In [14]:
!ls

Best_Xception_12_epochs.hdf5
deepAgeGenClassification.py
del.py
F-metrics.ipynb
garbage.py
load-predict.ipynb
modelResNetM.py
model_results_128.ipynb
model_results_200.ipynb
model_results_71.ipynb
my_Xception.py
results3.txt
results4.txt
weights-improvement-01-0.91.hdf5
weights-improvement-01-0.91_new.hdf5
weights-improvement-02-0.93.hdf5
weights-improvement-03-0.93.hdf5
weights-improvement-04-0.95.hdf5
weights-improvement-05-0.95.hdf5
weights-improvement-06-0.95.hdf5
weights-improvement-07-0.96.hdf5
weights-improvement-08-0.95.hdf5
weights-improvement-09-0.96.hdf5
weights-improvement-10-0.96.hdf5
weights-improvement-11-0.95.hdf5
weights-improvement-12-0.97.hdf5
weights-improvement-128-01-0.91.hdf5
weights-improvement-128-02-0.94.hdf5
weights-improvement-128-03-0.94.hdf5
weights-improvement-128-04-0.94.hdf5
weights-improvement-128-05-0.94.hdf5
weights-improvement-128-06-0.95.hdf5
weights-improvement-128-07-0.95.hdf5
weights-improvement-128-08-0.96.hdf5
weights-improvement-128-09-0.96.h